# xcube JupyterLab Integration

This Notebook demonstrates how to use the features of the xcube JupyterLab integration.
The Notebook demonstrates two scenarios how xcube Viewer is utilized in JupyterLab.
In particular, we open xcube Viewer for any `xarray.Dataset` instances 

1. opened or otherwise created in this Notebook (in-memory datasets);
2. persisted in this Notebooks's workspace or from other sources (saved datasets).

For this to work, the following requirements must be satisfied:

* xcube version must be >= 0.13
* JupyterLab version must be 3.4.x
* The xcube JupyterLab extension [xcube-jl-ext](https://github.com/dcs4cop/xcube-jl-ext) 
  must be installed and enabled.

If you run JupyterLab remotely, e.g., with JupyterHub, then also

* the [jupyter-proxy](https://jupyter-server-proxy.readthedocs.io/en/latest/#) 
  server extension must be installed and enabled.
  

In [1]:
import numpy as np
import xarray as xr

from xcube.webapi.viewer import Viewer
from xcube.core.store import new_data_store
from xcube.core.select import select_subset

We create some datasets so we have something to show. We use the xcube datastore framework here to open the dataset, but it could also be opened by other means, e.g., `xr.open_dataset()`, provided it has variables with dimensions ["time", "y", "x"] or ["y", "x"]. 

In [2]:
store = new_data_store("s3", root="esdl-esdc-v2.1.1", storage_options={"anon": True})
store.list_data_ids()

['esdc-8d-0.083deg-184x270x270-2.1.1.zarr',
 'esdc-8d-0.083deg-1x2160x4320-2.1.1.zarr',
 'esdc-8d-0.25deg-184x90x90-2.1.1.zarr',
 'esdc-8d-0.25deg-1x720x1440-2.1.1.zarr']

In [3]:
dataset = store.open_data('esdc-8d-0.083deg-1x2160x4320-2.1.1.zarr')

In [4]:
dataset_subset = select_subset(
    dataset,
    bbox=[6, 48, 14, 54],
    time_range=["2016-01-01 00:00:00", 
                "2018-12-31 23:59:59"],
    var_names=['evaporation', 'ozone', 'cfc', 'air_temperature_2m']
)
dataset_subset.attrs["title"] = "My ESDC Subset"
dataset_subset

<xarray.Dataset>
Dimensions:             (time: 138, lat: 72, lon: 96, bnds: 2)
Coordinates:
  * lat                 (lat) float64 53.96 53.88 53.79 ... 48.21 48.12 48.04
    lat_bnds            (lat, bnds) float64 dask.array<chunksize=(72, 2), meta=np.ndarray>
  * lon                 (lon) float64 6.042 6.125 6.208 ... 13.79 13.87 13.96
    lon_bnds            (lon, bnds) float64 dask.array<chunksize=(96, 2), meta=np.ndarray>
  * time                (time) datetime64[ns] 2016-01-05 ... 2018-12-31
    time_bnds           (time, bnds) datetime64[ns] dask.array<chunksize=(138, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    air_temperature_2m  (time, lat, lon) float32 dask.array<chunksize=(1, 72, 96), meta=np.ndarray>
    cfc                 (time, lat, lon) float32 dask.array<chunksize=(1, 72, 96), meta=np.ndarray>
    evaporation         (time, lat, lon) float32 dask.array<chunksize=(1, 72, 96), meta=np.ndarray>
    ozone               (time, lat, lon) float32 dask.array<chunksize=(1, 72, 96), meta=np.ndarray>
Attributes: (12/35)
    Metadata_conventions :     Unidata Dataset Discovery v1.0
    acknowledgment:            The ESDL team acknowledges all data providers! 
    chunking:                  1x2160x4320
    comment:                   none. 
    contributor_name:          Max Planck Institute for Biogeochemistry
    contributor_role:          ESDL Science Lead 
    ...                        ...
    summary:                   This data set contains a data cube of Earth Sy...
    time_coverage_duration:    P40Y
    time_coverage_end:         31.12.2018
    time_coverage_resolution:  P8D
    time_coverage_start:       05.01.1979
    title:                     My ESDC Subset

---
**Scenario 1**: Open xcube Viewer for a dataset instances opened or otherwise created in this Notebook (in-memory datasets).

In [5]:
viewer = Viewer()

In [6]:
viewer.add_dataset(dataset)
viewer.add_dataset(dataset_subset)

'1c48f3f9-5e74-4005-8361-25797740d806'

You can click on the viewer link to open xcube Viewer in a new browser tab:

In [7]:
viewer.info()

Server: http://localhost:8888/proxy/8000
Viewer: http://localhost:8888/proxy/8000/viewer/?serverUrl=http://localhost:8888/proxy/8000


You can also open xcube Viewer inlined here:

In [8]:
viewer.show()

404 GET /viewer/config/config.json (127.0.0.1) 0.50ms


---
**Scenario 2**: Open xcube Viewer for a dataset instances persisted in this Notebooks's workspace or from other sources (saved datasets).

In [9]:
dataset_subset.to_zarr("esdc-subset.zarr", mode="w")

2023-02-07 14:10:04,168 - tornado.access - WARNING - 404 GET /viewer/config/config.json (127.0.0.1) 0.50ms
c:\users\norma\projects\xcube\xcube\util\timeindex.py:158: UserWarning: We can't determine whether the time indexer has a time zone. We will therefore omit the check on whether the time indexer and the variable are incompatible in terms of timezone awareness. This may result in an error when an indexing operation is carried out. If such an error occurs after this warning, make sure that the time indexer has timezone information.
  warnings.warn(
C:\Users\norma\mamba\envs\xcube\lib\site-packages\xarray\core\indexes.py:211: FutureWarning: Indexing a timezone-naive DatetimeIndex with a timezone-aware datetime is deprecated and will raise KeyError in a future version.  Use a timezone-naive object instead.
  indexer = index.slice_indexer(


C:\Users\norma\mamba\envs\xcube\lib\site-packages\rasterio\features.py:362: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _rasterize(valid_shapes, out, transform, all_touched, merge_alg)
C:\Users\norma\mamba\envs\xcube\lib\site-packages\rasterio\features.py:362: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _rasterize(valid_shapes, out, transform, all_touched, merge_alg)


Now, open the JupyterLab Launcher, and select _xcube Viewer_. You should see the dataset titled "My ESDC Subset". 

You should now find two new files in your workspace, [xcube-server.yaml](./xcube-server.yaml) and [xcube-server.log](./xcube-server.log). 
The first one is the configuration file used by xcube Server and therefore determines what datasets are shown in xcube Viewer. 
The second is the server's log file which is useful for debugging, for example, you may find details here why a certain dataset is not displayed.  

Note, currently [xcube-server.yaml](./xcube-server.yaml) only applies to the server/viewer launched by the JupyterLab Launcher. It is independent from the server/viewer created from within Notebooks.